# Module 7: Deep Learning and Image Recognition

For this exercise we will be using tensorflow. Tensorflow is a tensor computation framework. The Keras library is build on top of tensorflow to provide a much more user friendly interface.

Finally, to illustrate the ease of use of pretrained deep learning models, we will apply the fastai library on medical data.

## **Setup**
First install the necessary libraries.

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow tensorflow-datasets pydicom kornia opencv-python scikit-image pyarrow

In [ ]:
# Import Keras and Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### **Datasets**
Download the datasets, which might take a while.

In [ ]:
# This code block downloads the cats vs. dogs dataset

import tensorflow_datasets as tfds

# Load the cats vs. dogs dataset
train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:20%]", "train[20%:30%]", "train[30%:40%]"],
    as_supervised=True,  # Include labels
)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

In [ ]:
# This code loads the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

### **Exercise 7.1: Densely connected Neural Networks**

Use the sequential model from keras to build a Neural Network with only densely connected layers.
The model should be trained on the MNIST dataset and is finally evaluated on the test set.

In [ ]:
# Build a really simple densely connected neural network
model = None # TODO

In [ ]:
# look at the model with
model.summary()

In [ ]:
# Next we have to compile our model
# Choose an optimizer, a loss function and a metric to monitor
# TODO

In [ ]:
# Fit the model on the data
# Tip: Keep the number of epochs low, to reduce the runtime (e.g.: 2-4 epochs should suffice for a reasonable result)
# TODO

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = None # TODO
# What is the best performance you were able to achieve?
print("test loss, test acc:", results)

### **Exercise 7.2: Convolutional Neural Networks**

This time build a Convoluational Neural Network on the same MNIST dataset. Compare the test accuracy, training time and number of parameters between the two models. Which architecture would you use for the MNIST dataset?

In [ ]:
# The data needs to be 3D (pixel_height, pixel_width, color_channels/dimensions)
# That's why we run the code again
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# This time make the dataset 4-D
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [ ]:
# Build a Convolutional model
model = None # TODO

In [ ]:
# Look at the number of parameters with
model.summary()

In [ ]:
# Again, compile the model
# Choose an appropriate optimizer, loss function and metric to monitor
# TODO

In [ ]:
# Fit the model on the data
# Tip: Again, limit the number of epoch the algorithms runs to make it comparable to the previous experiment
# TODO

### **Exercise 7.3: Transferlearning**

For this exercise we will try to build a dog vs. cat predictor. Since the problem is much more complex than the previous one, we want to use a pre-trained image classification model.

Use the pre-trained ResNet-50 model and add a new top layer to classify only dogs and cats. Train the model for a couple of epochs and evaluate it on the test set.

### **Preprocessing**

In [ ]:
import matplotlib.pyplot as plt

# Plot a couple of the images
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(train_ds.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(int(label))
    plt.axis("off")

In [ ]:
# Standardization
# Rescale and fix the image size to 150,150
size = (150, 150)

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

In [ ]:
# Add caching and pre-loading for speedup
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

In [ ]:
# Use the pretrained ResNet-50 as a base_model
base_model = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    pooling='avg',
    input_shape=(150, 150, 3)
)

# Freeze the base model, by setting base_model to not trainable
# TODO

In [ ]:
# Create a new input layer for our new dataset
inputs = None # TODO

# Add data augmentation
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)
x = data_augmentation(inputs)

# Rescale color values of 0..255 to range of -1..+1
scale_layer = layers.Rescaling(scale=1/127.5, offset=-1)
ax = scale_layer(x)

# Put ResNet into inference mode
x = base_model(x, training=False)
# Add dropout to the final layer
x = keras.layers.Dropout(0.2)(x)

# Add another Dense layer on top of the base model
outputs = None # TODO

# Create the new model 
model = keras.Model(inputs, outputs)

In [ ]:
# Print the layers with
model.summary()

In [ ]:
# Train the top layer
# Choose an appropriate optimizer, loss and monitoring metric
# TODO

In [ ]:
# Fit the model
# TODO

In [ ]:
# Evaluate the model on the test set
# TODO

## **Bonus Exercise 7.4: Doing it the fast way**

In this exercise we use the fastai library to quickly finetune a pre-trained model.

In [ ]:
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.medical.imaging import *

import pydicom

import pandas as pd

In [ ]:
# Downloads the DICOM files
pneumothorax_source = untar_data(URLs.SIIM_SMALL)

In [ ]:
# Create a train and validation split
items = get_dicom_files(pneumothorax_source/f"train/")
trn,val = RandomSplitter()(items)

In [ ]:
# We can look at a sample
patient = 7
xray_sample = items[patient].dcmread()
xray_sample.show()

In [ ]:
# Read in the label information
df = pd.read_csv(pneumothorax_source/f"labels.csv")
df.head()

In [ ]:
# Put the training in a DataBlock
# That is later used as a data source for training
pneumothorax = DataBlock(blocks=(ImageBlock(cls=PILDicom), CategoryBlock),
                   get_x=lambda x:pneumothorax_source/f"{x[0]}",
                   get_y=lambda x:x[1],
                   batch_tfms=[*aug_transforms(size=224),Normalize.from_stats(*imagenet_stats)])

dls = pneumothorax.dataloaders(df.values, num_workers=0)

In [ ]:
# We can plot the first Batch of 16 images
dls = pneumothorax.dataloaders(df.values)
dls.show_batch(max_n=16)

In [ ]:
# Create the model
# For fastai you don't need to specify a loss function or optimizer
learn = cnn_learner(dls, resnet34, metrics=accuracy)

In [ ]:
# The loss function was chosen based on the dataset
# You can have a look at it with
learn.loss_func

In [ ]:
# The optimzer was chosen based on the dataset
# You can have a look at it with
learn.opt_func

In [ ]:
# Find the best learning rate
learn.lr_find()

In [ ]:
# Finetune the model
learn.fit_one_cycle(1)

In [ ]:
# Show the result of a single batch
learn.show_results(max_n=16)

In [ ]:
# Evaluate the performance with the confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(7,7))

In [ ]:
upp, low = interp.confusion_matrix()
tn, fp = upp[0], upp[1]
fn, tp = low[0], low[1]
print(tn, fp, fn, tp)

In [ ]:
# Calculate the sensitivity
# TODO

In [ ]:
# Calculate the specificity
# TODO

In [ ]:
# Calculate the accuracy
#TODO

**This is model considered good? How can we improve the model?**